In [29]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import classification_report
from imblearn.over_sampling import ADASYN, BorderlineSMOTE
from joblib import dump
from sklearn.ensemble import GradientBoostingClassifier, HistGradientBoostingClassifier

Loading of data

In [30]:
df = pd.read_csv('training_data.csv')

# Feature Engineering

Below are the list of feature engineering tried on the model, some are ignored after analysis

In [31]:
# df['procedures_per_day'] = df['num_procedures'] / df['time_in_hospital']
# df['medications_per_day'] = df['num_medications'] / df['number_diagnoses']
# df['stay_length'] = pd.cut(df['time_in_hospital'], bins=[0, 3, 7, 14, np.inf], labels=['Short', 'Medium', 'Long', 'Very Long'])
# df['procedure_intensity'] = pd.cut(df['num_procedures'], bins=[0, 1, 3, 5, np.inf], labels=['Very Low', 'Low', 'Medium', 'High'])
# df['high_num_procedures'] = df['num_procedures'].apply(lambda x: 1 if x > 3 else 0)  


df['total_visits'] = df['number_outpatient'] + df['number_emergency'] + df['number_inpatient']
df['high_number_diagnoses'] = df['number_diagnoses'].apply(lambda x: 1 if x > 8 else 0)
df = df.drop(['number_diagnoses','number_inpatient','number_outpatient','number_emergency','change'], axis=1)


# Pre-processing

In [33]:
# Preprocess categorical features
categorical_features = ['admission_type_id','discharge_disposition_id','medical_specialty','diag_1']
le = LabelEncoder()
for feature in categorical_features:
    df[feature] = le.fit_transform(df[feature])


# Custom train_test_split based on patient_nbr
def custom_train_test_split(df, patient_col='patient_nbr', test_size=0.3):
    patients = df[patient_col].unique()
    train_patients, test_patients = train_test_split(patients, test_size=test_size, random_state=42)
    train_data = df[df[patient_col].isin(train_patients)]
    test_data = df[df[patient_col].isin(test_patients)]
    return train_data, test_data

Custom train-test split based on patient number is to get unique data in train and test sample. This is to make the model predict on un-seen data.

In [34]:
train_data, test_data = custom_train_test_split(df)

# Split features and target
X_train = train_data.drop(columns=['readmitted', 'patient_nbr'])
y_train = train_data['readmitted']
X_test = test_data.drop(columns=['readmitted', 'patient_nbr'])
y_test = test_data['readmitted']


Applying ADASYN to the imbalanced training set. it creates synthetic samples in minority class.

ADASYN is ignored after model evaluation as this can lead to overfitting. 

In [ ]:
# adasyn = ADASYN(sampling_strategy='auto', random_state=42)
# X_train_resampled, y_train_resampled = adasyn.fit_resample(X_train, y_train)

Handle class imbalance using class weights, which seems to be better option than creating synthetic samples.

In [35]:
class_weights = dict(pd.Series(y_train).value_counts(normalize=True).apply(lambda x: 1/x).to_dict())


# Hyper-parameter tuning

Hyper-parameter tuning with GridSearchCV

In [ ]:
# param_grid = {
#     'n_estimators': [10, 100, 200],
#     'max_depth': [None, 10, 20],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4]
# }

Trying different models with the best parameters

Gradient boosting model is ignored as it had issues with github

In [ ]:
# clf = GradientBoostingClassifier(
#     # bootstrap=False,
#     # class_weight='balanced',
#     random_state=42,
#     max_depth=10,
#     min_samples_leaf=4,
#     min_samples_split=10,
#     n_estimators=200
#     # class_weight=class_weights
# )

Random forest model is the final selected model

In [37]:
clf = RandomForestClassifier(
   random_state=42,
   max_depth=10,
   min_samples_leaf=4,
   min_samples_split=10,
   n_estimators=200,
   class_weight=class_weights
)

In [21]:
# clf = DecisionTreeClassifier(
#    random_state=42,
#    max_depth=10,
#    min_samples_leaf=4,
#    min_samples_split=10,
#    # n_estimators=200,
#    class_weight=class_weights
# )

Decision Tree gives more correct predictions for minority class, but the prediction on majority class becomes worse.

In [ ]:
# clf = HistGradientBoostingClassifier(
#     # bootstrap=False,
#     # class_weight='balanced',
#     random_state=42,
#     max_iter=200,
#     max_leaf_nodes=None
#     # max_depth=10,
#     # min_samples_leaf=1,
#     # min_samples_split=2,
#     # n_estimators=200
# )

Grid search using ROC-AUC as the evaluation metric

Used ROC-AUC as evaluation metric, because ROC-AUC is less affected by class imbalance compared to accuracy. Accuracy can be misleading in imbalanced datasets because predicting the majority class can yield high accuracy but poor performance in identifying the minority class.

In [ ]:
# grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, scoring='roc_auc', n_jobs=-1, verbose=2)
# grid_search.fit(X_train, y_train)

Best parameters are then printed

In [ ]:
# best_params = grid_search.best_params_
# print("Best parameters:", best_params)

Training the model with the best parameters

In [ ]:
# clf = RandomForestClassifier(**best_params, class_weight=class_weights)

In [38]:
clf.fit(X_train, y_train)

RandomForestClassifier(class_weight={0: 1.126105382587513,
                                     1: 8.929875628473141},
                       max_depth=10, min_samples_leaf=4, min_samples_split=10,
                       n_estimators=200, random_state=42)

# Evaluation

In [40]:
cv_scores = cross_val_score(clf, X_train, y_train, cv=5, scoring='roc_auc')
print("Cross-validation ROC-AUC scores:", cv_scores)
print("Mean cross-validation ROC-AUC score:", np.mean(cv_scores))

Cross-validation ROC-AUC scores: [0.63134451 0.66305821 0.65419095 0.64837116 0.64837858]
Mean cross-validation ROC-AUC score: 0.6490686797872682


Make predictions

Predict the probabilities for the positive class (1) using the test data.


In [39]:
y_pred = clf.predict(X_test)
y_pred_proba = clf.predict_proba(X_test)[:, 1]

Evaluate model performance using ROC-AUC

In [41]:
roc_auc = roc_auc_score(y_test, y_pred_proba)
print("ROC-AUC:", roc_auc)

ROC-AUC: 0.6542918566827735


Confusion matrix

In [42]:
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)

Confusion Matrix:
 [[17482  8298]
 [ 1550  1820]]


Precision, Recall, F1-score

In [43]:
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)

Precision: 0.8331601319648946
Recall: 0.6621612349914237
F1-Score: 0.7212343523632785


Recall = 0.662, This means that approximately 66.2% of the actual high-risk patients were correctly identified by the model.

The F1-Score is a measure of a model's accuracy on a dataset and is especially useful when you have an imbalanced dataset. It is the harmonic mean of precision and recall, providing a balance between the two. An F1-Score of 0.7212343523632785 indicates that the model has a reasonably good balance between precision and recall, though it's not perfect. 

# Feature importances

In [18]:
feature_importances = pd.Series(clf.feature_importances_, index=X_train.columns).sort_values(ascending=False)
print("Feature Importances:\n", feature_importances)

Feature Importances:
 total_visits                0.258334
discharge_disposition_id    0.177664
diag_1                      0.164199
num_medications             0.120224
time_in_hospital            0.086875
medical_specialty           0.067337
num_procedures              0.053615
admission_type_id           0.045093
high_number_diagnoses       0.026660
dtype: float64


Some of the least important features are removed after analyzing

Identifying a subset of patients in the test set who are at high risk and who the model predicts as having a high probability of readmission (greater than 0.8)

In [44]:
high_risk_patients = X_test[(y_test == 1) & (y_pred_proba > 0.8)]
print("High-risk patients predicted with high probability of readmission:\n", high_risk_patients)


High-risk patients predicted with high probability of readmission:
        admission_type_id  discharge_disposition_id  time_in_hospital  \
63170                  0                         1                 2   
64705                  0                         1                 2   

       medical_specialty  num_procedures  num_medications  diag_1  \
63170                 18               1               12     707   
64705                 18               1               11     707   

       total_visits  high_number_diagnoses  
63170             6                      0  
64705             8                      0  
